# Sales Prediction for Time Series Data

## Part 4: Models (Linear Models)

For the linear models, the first step is to see which scaler performs the best.  After selecting the scaler, comparisons will be made between the linear regression with no regularization, ridge regression, and elastic net.

Submissions are evaluated by root mean squared error (RMSE). True target values are clipped into [0,20] range.

In [1]:
import numpy as np
import pandas as pd 
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import lightgbm as lgb
import os
import time 
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import random
import gc

%matplotlib inline 

In [2]:
y_train = joblib.load('../data/y_train_lev_1.pkl')
y_test = joblib.load('../data/y_val_lev_1.pkl')
y_train_full = joblib.load('../data/y_train_full.pkl')

In [3]:
def submission(model, X_test):
    '''
    make submission file
    arguments:    model = model name 
                  X_test= X_test name
    return: a file saved in directory with timestamp
    '''
    # model prediction
    pred = model.predict(X_test)
    print('mean before clipping: ', pred.mean())
    pred = pred.clip(0,20)
    print('mean after clipping: ', pred.mean())

    # create prediction dataframe
    ID = joblib.load('ID.pkl')
    predDF = pd.DataFrame() 
    predDF['ID'] = ID
    predDF['item_cnt_month'] = pred
    print(predDF.head())

    # write dataframe to csv
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%m%d_%H.%M')
    print('submission_' + st + '.csv')
    
    predDF.to_csv(header=True, index=False, path_or_buf = 'submission_' + st + '.csv')
    
    return None

In [4]:
def linearReg(X_train, y_train, X_test, y_test):
    X_train = X_train.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
    X_test = X_test.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
    model = linear_model.LinearRegression()
    model.fit(X_train.values, y_train)
    pred = model.predict(X_test.values)
    pred = pred.clip(0, 20)

    MSETest = mean_squared_error(y_test, pred)
    RMSETest = np.sqrt(MSETest)
    
    MSETrain = mean_squared_error(y_train, model.predict(X_train.values).clip(0, 20))
    RMSETrain = np.sqrt(MSETrain)
    
    print('Train RMSE:', RMSETrain)
    print('Test RMSE:', RMSETest)
    print('')
    print('y_test mean: ', y_test.mean())
    print('prediction mean: ', pred.mean())
    
    return model

### Compare different scalers

Different scaler are used to train the linear model.    

#### No Scaling

In [5]:
X_train = pd.read_pickle('../data/X_train_lev_1_noScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_noScaler')

In [6]:
model = linearReg(X_train, y_train, X_test, y_test)

Train RMSE: 0.9041779879022737
Test RMSE: 1.2955651310937322

y_test mean:  0.33427927
prediction mean:  0.27298765614518306


In [7]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_no_scale_' + st + '.pkl')
print('lr_no_scale_' + st + '.pkl')

lr_no_scale_0201_15.05.pkl


#### Standard Scaler

In [8]:
X_train = pd.read_pickle('../data/X_train_lev_1_standardScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_standardScaler')

In [9]:
model = linearReg(X_train, y_train, X_test, y_test)

Train RMSE: 0.904214
Test RMSE: 1.1546773

y_test mean:  0.33427927
prediction mean:  0.25371638


In [10]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_standard_scale_' + st + '.pkl')
print('lr_standard_scale_' + st + '.pkl')

lr_standard_scale_0201_15.05.pkl


#### RobustScaler

In [11]:
X_train = pd.read_pickle('../data/X_train_lev_1_RobustScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_RobustScaler')

In [12]:
model = linearReg(X_train, y_train, X_test, y_test)

Train RMSE: 0.9041888
Test RMSE: 1.1566713

y_test mean:  0.33427927
prediction mean:  0.24483177


In [13]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_robust_scale_' + st + '.pkl')
print('lr_robust_scale_' + st + '.pkl')

lr_robust_scale_0201_15.05.pkl


#### MinMax Scaler

In [14]:
X_train = pd.read_pickle('../data/X_train_lev_1_MinMaxScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_MinMaxScaler')

In [15]:
model = linearReg(X_train, y_train, X_test, y_test)

Train RMSE: 0.9041996
Test RMSE: 1.1542825

y_test mean:  0.33427927
prediction mean:  0.25732833


In [16]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_minmax_scale_' + st + '.pkl')
print('lr_minmax_scale_' + st + '.pkl')

lr_minmax_scale_0201_15.06.pkl


#### Quantile Scaler

In [17]:
X_train = pd.read_pickle('../data/X_train_lev_1_QuantileScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_QuantileScaler')

In [18]:
model = linearReg(X_train, y_train, X_test, y_test)

Train RMSE: 1.014003
Test RMSE: 1.2312081

y_test mean:  0.33427927
prediction mean:  0.31087527


In [19]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_quantile_scale_' + st + '.pkl')
print('lr_quantile_scale_' + st + '.pkl')

lr_quantile_scale_0201_15.06.pkl


### Compare with Ridge Regression and Elastic Net

#### Ridge Regression - L2 regularization
Alpha of 0 corresponds to linear regression.  

In [20]:
X_train = pd.read_pickle('../data/X_train_lev_1_MinMaxScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_MinMaxScaler')
X_train = X_train.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
X_test = X_test.drop(['shop_id','item_id', 'item_category_id'], axis = 1)

In [21]:
ridgeResult = {}
for alpha in [0, 1e-10, 1e-5, 1e-3, 1e-2, 0.5, 1, 10]:    
    # define model
    model = linear_model.Ridge(alpha = alpha, normalize = True)
    model.fit(X_train.values, y_train)
    
    #make prediction
    pred = model.predict(X_test.values)
    pred = pred.clip(0, 20)

    # calculate score
    MSETest = mean_squared_error(y_test, pred)
    RMSETest = np.sqrt(MSETest)
    MSETrain = mean_squared_error(y_train, model.predict(X_train.values))
    RMSETrain = np.sqrt(MSETrain)
    
    # save results
    ridgeResult[alpha] = RMSETest
    
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
    joblib.dump(model, 'ridge_' + str(alpha) + '_' + st + '.pkl')
    
    print('ridge_' + str(alpha) + '_' + st + '.pkl')
    print('Alpha:', alpha, 'Train RMSE:', RMSETrain, 'Test RMSE:', RMSETest)
    

ridge_0_0201_15.06.pkl
Alpha: 0 Train RMSE: 0.90642697 Test RMSE: 1.1543889
ridge_1e-10_0201_15.06.pkl
Alpha: 1e-10 Train RMSE: 0.9063489 Test RMSE: 1.3144988
ridge_1e-05_0201_15.07.pkl
Alpha: 1e-05 Train RMSE: 0.90634835 Test RMSE: 1.1547109
ridge_0.001_0201_15.07.pkl
Alpha: 0.001 Train RMSE: 0.9063636 Test RMSE: 1.1545651
ridge_0.01_0201_15.07.pkl
Alpha: 0.01 Train RMSE: 0.9065299 Test RMSE: 1.1544814
ridge_0.5_0201_15.07.pkl
Alpha: 0.5 Train RMSE: 0.92709523 Test RMSE: 1.1845108
ridge_1_0201_15.07.pkl
Alpha: 1 Train RMSE: 0.94325536 Test RMSE: 1.2006776
ridge_10_0201_15.07.pkl
Alpha: 10 Train RMSE: 1.0806948 Test RMSE: 1.3050978


In [22]:
print('The alpha with best RMSE: ', min(ridgeResult, key=ridgeResult.get), 'RMSE:', ridgeResult[min(ridgeResult, key=ridgeResult.get)])

The alpha with best RMSE:  0 RMSE: 1.1543889


#### Elastic Net

l1_ratio = 0 the penalty is an L2 penalty. 
<br> For l1_ratio = 1 it is an L1 penalty. 
<br>For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.

In [23]:
X_train = pd.read_pickle('../data/X_train_lev_1_MinMaxScaler')
X_test = pd.read_pickle('../data/X_test_lev_1_MinMaxScaler')
X_train = X_train.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
X_test = X_test.drop(['shop_id','item_id', 'item_category_id'], axis = 1)

In [24]:
for l1_ratio in [0.01, 0.05, .1, .5, .7, .9, 1]:
    model = linear_model.ElasticNet(alpha = 1,
                                    l1_ratio = l1_ratio, 
                                    random_state = 123)
    model.fit(X_train.values, y_train)
    #make prediction
    pred = model.predict(X_test.values)
    pred = pred.clip(0, 20)

    # calculate score
    MSETest = mean_squared_error(y_test, pred)
    RMSETest = np.sqrt(MSETest)
    MSETrain = mean_squared_error(y_train, model.predict(X_train.values))
    RMSETrain = np.sqrt(MSETrain)
    
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
    joblib.dump(model, 'elasticNet_' + str(l1_ratio) + '_' + st + '.pkl')
    
    print('elasticNet_' + str(l1_ratio) + '_' + st + '.pkl')
    print('l1_ratio:', l1_ratio, 'Train RMSE:', RMSETrain, 'Test RMSE:', RMSETest)

elasticNet_0.01_0201_15.07.pkl
l1_ratio: 0.01 Train RMSE: 1.2937804 Test RMSE: 1.4703438
elasticNet_0.05_0201_15.07.pkl
l1_ratio: 0.05 Train RMSE: 1.3061615 Test RMSE: 1.479641
elasticNet_0.1_0201_15.07.pkl
l1_ratio: 0.1 Train RMSE: 1.3065388 Test RMSE: 1.4799339
elasticNet_0.5_0201_15.07.pkl
l1_ratio: 0.5 Train RMSE: 1.3065388 Test RMSE: 1.4799339
elasticNet_0.7_0201_15.07.pkl
l1_ratio: 0.7 Train RMSE: 1.3065388 Test RMSE: 1.4799339
elasticNet_0.9_0201_15.08.pkl
l1_ratio: 0.9 Train RMSE: 1.3065388 Test RMSE: 1.4799339
elasticNet_1_0201_15.08.pkl
l1_ratio: 1 Train RMSE: 1.3065388 Test RMSE: 1.4799339


#### Ridge model for the full training set

In [25]:
X_train_full = pd.read_pickle('../data/X_train_MinMaxScaler')
X_test_full = pd.read_pickle('../data/X_test_MinMaxScaler')
X_train_full = X_train_full.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
X_test_full = X_test_full.drop(['shop_id','item_id', 'item_category_id'], axis = 1)

In [26]:
for alpha in [min(ridgeResult, key=ridgeResult.get)]:
    model = linear_model.Ridge(alpha = alpha, normalize = True, max_iter = 3000)
    model.fit(X_train_full.values, y_train_full)

In [27]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'ridge_full_' + st + '.pkl')
# ridge = joblib.load('ridge.pkl') 

['ridge_full_0201_15.10.pkl']

In [28]:
submission(model, X_test_full)

mean before clipping:  6470.0054
mean after clipping:  0.301176
   ID  item_cnt_month
0   0        0.402344
1   1        0.527344
2   2        0.808594
3   3        0.097656
4   4        1.144531
submission_0201_15.10.csv


#### Linear model for the full dataset
<br>The RMSE for this submission: 0.99202

In [33]:
X_train_full = pd.read_pickle('../data/X_train_RobustScaler')
X_test_full = pd.read_pickle('../data/X_test_RobustScaler')
X_train_full = X_train_full.drop(['shop_id','item_id', 'item_category_id'], axis = 1)
X_test_full = X_test_full.drop(['shop_id','item_id', 'item_category_id'], axis = 1)

In [34]:
model = linear_model.LinearRegression(n_jobs = 4)
model.fit(X_train_full, y_train_full)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=4, normalize=False)

In [35]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%m%d_%H.%M')
joblib.dump(model, 'lr_full_' + st + '.pkl')

['lr_full_0201_21.06.pkl']

In [36]:
submission(model, X_test_full)

mean before clipping:  0.2698322
mean after clipping:  0.29283786
   ID  item_cnt_month
0   0        0.422253
1   1        0.531743
2   2        0.857724
3   3        0.099128
4   4        1.141984
submission_0201_21.06.csv
